In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame([[8,8,4], [7,9,5], [6,10,6], [5,15,7]], columns=['cgpa', 'profile_score', 'lpa'])

In [3]:
df

,cgpa,profile_score,lpa
0,8,8,4
1,7,9,5
2,6,10,6
3,5,15,7


In [ ]:
# initializing parameters function for a deep neural network
# layer_dims is a list containing the number of neurons in each layer
def initialize_parameters(layer_dims):

  np.random.seed(3)
  parameters = {}
  L = len(layer_dims)

  for l in range(1, L):

    parameters['W' + str(l)] = np.ones((layer_dims[l - 1], layer_dims[l])) * 0.1
    parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

  return parameters

In [ ]:
initialize_parameters([2,2,1]) # testing the function

{'W1': array([[0.1, 0.1],
        [0.1, 0.1]]),
 'b1': array([[0.],
        [0.]]),
 'W2': array([[0.1],
        [0.1]]),
 'b2': array([[0.]])}

In [ ]:
# linear forward function for calculating output
def linear_forward(A_prev, W, b):

  Z = np.dot(W.T, A_prev) + b # Z = W.T * A_prev + b

  return Z

In [ ]:
# main function for forward propagation
def L_layer_forward(X, parameters):

  A = X
  L = len(parameters) // 2 # number of layers in the neural network

  for l in range(l + L+l):
    A_prev = A # input to the layer
    W1 = parameters['W' + str(l)] # weights 